## scoring function

In [1]:
def scoring(data):
    from sklearn.preprocessing import LabelEncoder
    import category_encoders as ce
    from copy import deepcopy
    from sklearn.preprocessing import MinMaxScaler
    import pickle
    from sklearn.tree import DecisionTreeClassifier
    
    X = data.copy()
    
    def currencyCleaning(x):
        x = x[1:].replace(',','')
        return x
    data['DisbursementGross'] = data['DisbursementGross'].apply(currencyCleaning).astype(float).astype(int)
    data['SBA_Appv'] = data['SBA_Appv'].apply(currencyCleaning).astype(float).astype(int)
    data['BalanceGross'] = data['BalanceGross'].apply(currencyCleaning).astype(float).astype(int)
    data['GrAppv'] = data['GrAppv'].apply(currencyCleaning).astype(float).astype(int)

    data = data[data.NAICS != 0]
    data = data[(data.NewExist == 1.0) | (data.NewExist == 2.0)]
    data = data[(data.RevLineCr == 'Y') | (data.RevLineCr == 'N')]
    data = data[(data.LowDoc == 'Y') | (data.LowDoc == 'N')]
    data["Zip"] = data["Zip"].apply(str)
    
    
    
    
    loaded_scalers = pickle.load(open('scalers.sav', 'rb'))
    loaded_encoders = pickle.load(open('categorical_encoders.sav', 'rb'))
    loaded_model = pickle.load(open('decision_tree_model.sav', 'rb'))
    
    
    cols_to_drop = []
    cat_encoders = {}
    for col in data.drop(columns=['MIS_Status']).columns:
        if data[col].dtype == 'object':
            print("Target Encode scaling of ", col)
            enc = loaded_encoders[col][0]
            data[col+"_trg"] = enc.transform(data[[col]])
            cat_encoders[col] = [deepcopy(enc),"trg"]
            cols_to_drop.append(col)
            
    num_scalers = {}
    '''Scale only original columns'''
    for col in data.drop(columns=['MIS_Status']).columns[0:19]:
      if pd.api.types.is_numeric_dtype(data[col].dtype):
        print("MinMax scale of ", col)
        scaler = loaded_scalers[col][0]
        data[col+"_sc"] = scaler.transform(data[[col]])
    
        num_scalers[col] = [deepcopy(scaler),"MinMax"]
        cols_to_drop.append(col)
    
    X = data.drop(columns=['MIS_Status']+cols_to_drop)
    Y = data.MIS_Status
    
    y_predicted = loaded_model.predict(X)

    return list(y_predicted)

In [2]:
import pandas as pd

The function"Currency Cleaning" has been written to deal with "Object" data type for Disbursement gross, Balance gross, GrAppv, SBA_Appv--convert them to float/int.